# Use the `frcmod` and `mol2` files to solvate each host-guest system with `paprika`

In [88]:
import glob as glob
import os as os
import subprocess as sp
import numpy as np

import parmed as pmd
from paprika.align import zalign
from paprika.tleap import System

In [87]:
complexes = glob.glob('./OA-G*')

First, let's align the host along the *z* axis, so we will pull the guest towards +z. I'm going to align the vector connecting the center of mass of the oxygens on bottom ring of OA and the bridging oxygens on the top ring of OA.

In [102]:
for complx in complexes:
    system = pmd.load_file(os.path.join(complx, 'AMBER', 'complex.pdb'))
    aligned_system = zalign(system, '@O10,O11,O32,O26,O9,O24,O25,O12', 
                            '@O16,O17,O19,O18,O15,O14,O20,O13', save=True, 
                            filename=os.path.join(complx, 'AMBER', 'complex_aligned.pdb'))

<img src="complex_aligned.png">

In [106]:
for complx in complexes:
    system = System()
        
    system.output_path = os.path.join(complx, 'AMBER')
    system.target_waters = 2500
    system.output_prefix = 'solvate'
       
    system.neutralize = True
    system.pbc_type = 'rectangular'
    system.add_ions = ['Na+', '0.06M', 'Cl-', '0.06M']
    system.template_lines = ['source leaprc.water.tip3p',
                           'source leaprc.gaff',
                           'loadamberparams hst.frcmod',
                           'HST = loadmol2 hst.mol2',
                           'loadamberparams gst.frcmod',
                           'GST = loadmol2 gst.mol2',
                           'model = loadpdb complex_aligned.pdb'
    ]
    
    system.build()

In [107]:
for complx in complexes:
    structure = pmd.load_file(os.path.join(complx, 'AMBER', 'solvate.prmtop'))
    
    print(f'Total charge {pmd.tools.netCharge(structure).execute():0.3f} ...')
    
    sodium = len(structure[':Na+'].atoms)
    chloride = len(structure[':Cl-'].atoms)
    
    print(f'{sodium} Na+, {chloride} Cl- in solvated structure...')
    
    hst = pmd.load_file(os.path.join(complx, 'AMBER', 'hst.mol2'), structure=True)
    gst = pmd.load_file(os.path.join(complx, 'AMBER', 'gst.mol2'), structure=True)
    
    hst_gst = hst + gst
    hst_gst_charge = pmd.tools.netCharge(hst_gst).execute()
    
    if hst_gst_charge < 0:
        sodium_added_to_neuralize = np.ceil(abs(hst_gst_charge))
        chloride_added_to_neutralize = 0
    elif hst_gst_charge > 0:
        sodium_added_to_neuralize = 0
        chloride_added_to_neutralize = np.ceil(abs(hst_gst_charge))
    else:
        sodium_added_to_neuralize = 0
        chloride_added_to_neutralize = 0
    
    print(f'{sodium_added_to_neuralize} Na+, {chloride_added_to_neutralize} Cl- for neutralization...')

    
    counterion_mass = (sodium - sodium_added_to_neuralize) * 22.9898 + \
                      (chloride - chloride_added_to_neutralize) * 35.453
    
    AMU_TO_GRAMS = 1.66054 * 10**-24
    molar_concentration = counterion_mass * (AMU_TO_GRAMS) / (22.9898 + 35.453)

    volume = structure.get_box()[0] * structure.get_box()[1] * structure.get_box()[2]
    ANGSTROM_CUBED_TO_LITERS = 1 * 10 ** -27
    liters = volume * ANGSTROM_CUBED_TO_LITERS
    
    moles_per_liter = molar_concentration / liters
    print(f'Counterion molarity {moles_per_liter:0.3f} (beyond neutralization)...')

Total charge 0.000 ...
11 Na+, 2 Cl- in solvated structure...
9.0 Na+, 0 Cl- for neutralization...
Counterion molarity 0.031 (beyond neutralization)...
Total charge 0.000 ...
11 Na+, 2 Cl- in solvated structure...
10.0 Na+, 0 Cl- for neutralization...
Counterion molarity 0.025 (beyond neutralization)...
Total charge 0.000 ...
11 Na+, 2 Cl- in solvated structure...
9.0 Na+, 0 Cl- for neutralization...
Counterion molarity 0.031 (beyond neutralization)...
Total charge 0.000 ...
11 Na+, 2 Cl- in solvated structure...
9.0 Na+, 0 Cl- for neutralization...
Counterion molarity 0.031 (beyond neutralization)...
Total charge 0.000 ...
11 Na+, 2 Cl- in solvated structure...
10.0 Na+, 0 Cl- for neutralization...
Counterion molarity 0.025 (beyond neutralization)...
Total charge 0.000 ...
11 Na+, 2 Cl- in solvated structure...
10.0 Na+, 0 Cl- for neutralization...
Counterion molarity 0.025 (beyond neutralization)...
Total charge 0.000 ...
13 Na+, 4 Cl- in solvated structure...
9.0 Na+, 0 Cl- for neut